# S1 pulse shape analysis functions

Version 3.1
  * Mock waveforms introduced

Version 3.0
  * per-s1 smearing to account for offset t0 and digitizer bin
  * Sub-sample alignment using interpolation (does not influence results, but whatever)

Version 2.5, December 4, 2017 (yes, still 2017)
  * Bugfix for spe error (was only array for one channel...)
  * Adding SPE errors photon by photon now (quad_adding default True)
  * check_errors bugfix (no squares for chi2)
  

Version 2.4, December 1, 2017
  * Re-introducted error_pct in sigma computation
  * Added the data version of the spe error (though may be too large in current form...)
  * bugfix for n steps in produce_settings_dicts
  * Added plot_model_manual for already computed values :)
  * Added check_errors for computed values

Version 2.3, November 30, 2017
  * Energy selection implemented (also in default, default is set to 5-20 keV to be roughly compatible)
  * Dict splitting function
  * Taking out s1_sample functionality
  * Better S1 size array building in simulation (rather then just guessing)

Version 2.2, November 29, 2017
  * Splitting the errors in two parts, got rid of (minus, base, plus)
  * Bugfix for sigma_stat (error was first added, then quadratic adding)
  * Bugfix for sigma_syst (base model was computed twice)
  * Took out the different modes for chi2
  * Scheduled some depricated functions for removal
  * Modified default parameters to high statistic ones: 250 bootstrap, 2e6 photons.
  * stored_stat_value in parameters instead of globals
  
Version 2.1, November 27, 2017
  * Major cleanup, removing some of the functionality....
  * Including SPE error
  * Added bootstrap error in value in memory
  * Normalization of the average pulse within the time range

# Settings

In [1]:
pickle_dir = '/data/xenon/ehogenbi/pulsefit/pickles/'

# Imports

In [1]:
import numpy as np
import os
import pickle
import time

import scipy
from scipy import stats
from scipy import optimize
from scipy import interpolate
import scipy.integrate as integrate
from scipy.integrate import odeint
from copy import deepcopy
from multihist import Hist1d, Histdd

# Default settings

In [3]:
# Digitizer sample size
dt = 2
# time range. Note that cutting some of this out will also reduce the real data waveforms...
# These are aligned by their center index!
valid_t_range = (-100, 290)

spe_ts = np.linspace(0, 639*2, 640) - 340 * 2
# Valid time (because the waveform does not range the full time span)
t_mask = (valid_t_range[0] <= spe_ts) & (spe_ts < valid_t_range[1])
spe_ts = spe_ts[t_mask]
spe_t_edges = np.concatenate([[spe_ts[0] - dt/2], spe_ts + dt/2])

default_params = dict(
    t1 = 3.1,    # Singlet lifetime, Nest 2014 p2
    t3 = 24,     # Triplet lifetime, Nest 2014 p2
    fs = 0.2,    # Singlet fraction
    tts = 2.,     # Transit time spread.
    f_r = 0.,
    tr = 15,
    fs_r = 0.2,
    eta = 0.,
    
    s1_min = 0,
    s1_max = 600,
    e_min = 5,
    e_max = 20,
    dset = 'er',
    aft = 0.28, # 0.28
    n_photons = int(2e6),
    t_min = -10.,
    t_max = 125.,
    error_offset  = 1e-4 , 
    error_pct = 0.,
    neglect_statistical = False,
    neglect_systematic = False,
    s1_model = 'two_exp', # two_exp, recombination
    bootstrap_trials = 250,
    stored_stat = True,
    stored_stat_value = None,
    max_shift = 10,
)

def get_params(params):
    '''
    Returns full set of parameters, setting the values given in `params` and setting the values in 
    `default_params` if not set explicity.
    '''
    for k, v in default_params.items(): # key, value
        params.setdefault(k, v)
    if params['tts'] < 0:
        params['tts'] = 1e-6
    # Parameters that may not be smaller than zero
    for par in ['fs', 'aft', 'f_r', 'fs_r', 'eta']:
        if params[par] < 0:
            params[par] = 0
        elif params[par] >1:
            params[par] =1
    return params

## Load PMT pulses

## Pulse shape

### High resolution pulse

In [4]:
def rebin(y, nsamples):
#     if len(y) % nsamples != 0:
#         raise ValueError('No es possibile')
    nbins = int(len(y) / nsamples)
    return np.average(np.reshape(y[:nbins * nsamples], (nbins, nsamples)), axis=1)

In [5]:
dt_fine = 0.1
rebin_factor = 2

In [6]:
# Read the pickle file
spe_ys_fine = []
for ch in [0, 1]:
    fn = os.path.join(pickle_dir, 'highrespulse_ch%d.pickle' % ch)
    t_fine, ys = pickle.load(open(fn, 'rb'))
    spe_ys_fine.append(ys)

In [ ]:
# Errors
with open(os.path.join(pickle_dir, 'highrespulse_error.pickle'), 'rb') as f:
    _, spe_dys_fine = pickle.load(f)

In [7]:
# Rebin to speed up computation
t_fine = rebin(t_fine, rebin_factor)
spe_ys_fine = [rebin(ys, rebin_factor) for ys in spe_ys_fine]
spe_dys_fine = rebin(spe_dys_fine, rebin_factor)
dt_fine = dt_fine * rebin_factor

In [8]:
# Limiting the time range
spe_ts_fine = t_fine
t_mask = (valid_t_range[0] <= spe_ts_fine) & (spe_ts_fine < valid_t_range[1])
spe_ts_fine = spe_ts_fine[t_mask]
spe_t_edges_fine = np.concatenate([[spe_ts_fine[0] - dt_fine/2], spe_ts_fine + dt_fine/2])
spe_ys_fine = [ys[t_mask] for ys in spe_ys_fine]
spe_dys_fine = spe_dys_fine[t_mask]

# Normalization
spe_dys_fine = spe_dys_fine / (np.average(np.sum(spe_ys_fine, axis = 1))) * dt_fine / dt
spe_ys_fine = [ys / np.sum(ys) * dt_fine / dt for ys in spe_ys_fine]

In [9]:
def shift_samples(y, nsamples):
    '''
    Shift samples to the right (negative means left)
    '''
    if nsamples == 0:
        return y
    elif nsamples > 0:
        return np.concatenate([np.zeros(nsamples), y[:-nsamples]])
    elif nsamples < 0:
        return np.concatenate([y[ - nsamples:], np.zeros(-nsamples)])

## Gain variation

In [10]:
gain_params = []
for ch, fn in enumerate(['170323_103732', '170323_104831']):
    with open(os.path.join(pickle_dir, '%s_ch%d_function.pickle' % (fn, ch)) , 'rb') as infile:
        _norm, _popt, _perr = pickle.load(infile)
        gain_params.append(np.concatenate([np.array([_norm]), _popt, _perr]))
gain_params = np.array(gain_params)

In [11]:
def area_sample(n_values, gain_params, channel):
    norm, mu, sigma, _, _ = gain_params[channel]
    lower, upper = (0., 3.)
    X = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
    return X.rvs(n_values)

In [12]:
def gaus_trunc(x, mu, sigma):
    return (x > 0) * np.exp( - (x - mu)**2 / (2 * sigma**2))

# S1 model

## Recombination model

### Even better model

In [13]:
def n_eta(t, tr, eta):
    params = [tr, eta]
    y0 = [(1- eta), 1]
    # print(eta)
    
    # Define the differential equations to solve
    def f(y, t, params):
        n_minus, n_plus = y
        tau, eta  = params
        alpha = 1/tau
        derivs = [- alpha * n_minus * n_plus, - alpha * n_minus * n_plus]
        return derivs
    
    psoln = odeint(f, y0, t, args=(params,))
    # Return only the electron number...
    return psoln

def n_product(t, tr, eta):
    psoln = n_eta(t, tr, eta)
    n_e = psoln[:, 0]
    n_holes = psoln[:, 1]
    return 1/ tr * n_e * n_holes


def Ir3(t, tau, tr, eta, tmax, nsteps):
    t_fine = np.linspace(0, tmax, nsteps)
    pdf = np.exp(-t_fine/tau) / tau * np.cumsum(n_product(t_fine, tr, eta) * np.exp(t_fine/tau))
    return np.interp(t, t_fine, pdf)

def Ir3_cdf(t, tau, tr, eta, tmax, nsteps):
    pdf = Ir3(t, tau, tr, eta, tmax, nsteps)
    return np.cumsum(pdf) / sum(pdf)

def Ir3_cdf_inv(t, tau, tr, eta, tmax, nsteps):
    cdf = Ir3_cdf(t, tau, tr, eta, tmax, nsteps)
    # 0 in the cdf means at time zero 
    return interpolate.interp1d(cdf, t, fill_value=(0, np.inf), bounds_error=False)

def simulate_recombination_times2(nphotons, tau, tr, eta, tmax, nsteps):
    t = np.linspace(0, tmax, nsteps)
    return Ir3_cdf_inv(t, tau, tr, eta, tmax, nsteps)(np.random.rand(nphotons))

# Simulation

In [14]:
import numba

def split_s1_groups(x, n_x, areas, channels, **params):
    """Splits x into groups with uniform (s1_min, s1_max) elements, then return matrix of histograms per group.
    Returns: integer array (n_x, n_groups)
    n_x: number of possible values in x. Assumed to be from 0 ... n_x - 1
    s1_min: minimum S1 number of hits
    s1_max: maximum S1 number of hits
    """
    params = get_params(params) 
    # Obtain the distribution that we wish to sample from
    s1s_data   = xams_data[params['dset']]['s1']
    e_data = xams_data[params['dset']]['e_ces']
    s1s_data = s1s_data[(s1s_data >= params['s1_min']) & (s1s_data < params['s1_max']) & 
                            (e_data >= params['e_min']) & (e_data < params['e_max'])]
    
    # Build S1 size array until it is large enough
    s1_block_size = int(1000)
    margin = 1.1
    pe_per_s1 = np.array([])
    while np.sum(pe_per_s1) < margin * np.sum(areas):
        # Add S1 sizes per block and check if large enough
        pe_per_s1  = np.concatenate([pe_per_s1, np.random.choice(s1s_data, size=s1_block_size)])
    # This is the length of the array, should be an overestimate of the number of S1s
    n_s1_est = len(pe_per_s1)
    # Shift per S1
    # This shifts each S1 to a random 0.2 ns wide bin, so that we do not align on the zero point all the time
    # Basically, this parameter is the time between true interaction and a digitizer edge
    if params['max_shift'] != 0:
        shifts_per_s1 = np.random.randint(0, params['max_shift'], size = n_s1_est)
    else:
        shifts_per_s1 = np.zeros(n_s1_est, dtype=int)
    
    # These are two arrays for the two channels
    # i.e. these will later yield the top and bottom waveform
    result0 = np.zeros((n_x, n_s1_est), dtype=float)
    result1 = np.zeros((n_x, n_s1_est), dtype=float)
    s1_i = _split_s1_groups(x, pe_per_s1, result0, result1, areas, channels, shifts_per_s1)
    return result0[:,:s1_i - 1], result1[:,:s1_i - 1]

@numba.jit(nopython=True)
def _split_s1_groups(x, pe_per_s1, result0, result1, areas, channels, shifts_per_s1):
    # One of these days, I'm going to cut you into little pieces
    # x is time index array
    s1_i = 0
    for photon_i, (i, ch) in enumerate(zip(x, channels)):
        if pe_per_s1[s1_i] < 0:
            s1_i += 1
            continue
        if ch == 0:
            result0[i + shifts_per_s1[s1_i], s1_i] += areas[photon_i]
        if ch == 1:
            result1[i + shifts_per_s1[s1_i], s1_i] += areas[photon_i]
        pe_per_s1[s1_i] -= areas[photon_i]
    return s1_i 


def shift(x, n):
    """Shift the array x n samples to the right, adding zeros to the left."""
    if n > 0:
        return np.pad(x, (n, 0), mode='constant')[:len(x)]
    else:
        return np.pad(x, (0, -n), mode='constant')[-len(x):]



def simulate_s1_pulse(**params):
    # n_photons=int(2e5), 
    """Return (wv_matrix, time_matrix, t_shift vector) for simulated S1s, consisting of n_photons in total
    """
    params = get_params(params)
    n_photons = params['n_photons']

    ##
    # Make matrix (n_samples, n_waveforms) of pulse waveforms with various shifts
    ##
    wv_matrix_list = []
    for ch in [0, 1]:
        # This is a matrix filled with waveforms, ordered by their SHIFT.
        # So, these are all just model waveforms and will be selected later
        y = spe_ys_fine[ch]
        i_noshift = np.searchsorted(spe_t_edges_fine, [0])[0]    # Index corresponding to no shift in the waveform
        wv_matrix = np.vstack([rebin(shift_samples(y, i - i_noshift), int(dt / dt_fine))
                           for i in range(len(spe_ts_fine))]).T 
        wv_matrix_list.append(wv_matrix)
    
    # Include SPE errors
    # Assumed the same for both channels...
    y = spe_dys_fine
    i_noshift = np.searchsorted(spe_t_edges_fine, [0])[0]    # Index corresponding to no shift in the waveform
    wv_err_matrix = np.vstack([rebin(shift_samples(y, i - i_noshift), int(dt / dt_fine))
                           for i in range(len(spe_ts_fine))]).T 
    
    ##
    # Simulate S1 pulse times, convert to index
    ##

    # Channel selector
    n_top = np.random.binomial(n=n_photons, p=params['aft']) # Number of photons happening in top array
    # First all the top channels, then all the bottom channels (ch1)
    channels = np.concatenate([np.zeros(n_top, dtype=int), np.ones(n_photons - n_top, dtype=int)])
    areas = np.concatenate([area_sample(n_top, gain_params, channel=0), 
                            area_sample(n_photons - n_top, gain_params, channel=1)])
    # Shuffle the two lists the exact same way
    channels, areas = unison_shuffled_copies(channels, areas)
    
    # Time is distributed according to exponential distribution
    # This is the TRUE time of all the photons generated, assuming time=0  is the time of the interaction
    times = np.zeros(n_photons)

    if params['s1_model'] == 'two_exp':
        n_singlets = np.random.binomial(n=n_photons, p=params['fs']) # We randomly select if the photon came from a 
                                                                     # singlet or triplet decay
        times += np.concatenate([
            np.random.exponential(params['t1'], n_singlets),
            np.random.exponential(params['t3'], n_photons - n_singlets)
        ])
    elif params['s1_model'] == 'recombination':
        n_recombination = np.random.binomial(n=n_photons, p=params['f_r'])
        n_recombination_singlets = np.random.binomial(n=n_recombination, p=params['fs_r'])
        n_recombination_triplets = n_recombination - n_recombination_singlets
        n_direct = n_photons - n_recombination
        n_direct_singlets = np.random.binomial(n=n_direct, p=params['fs'])
        n_direct_triplets = n_direct - n_direct_singlets
        assert (n_recombination_singlets + n_recombination_triplets + n_direct_singlets + n_direct_triplets == n_photons)        
        times += np.concatenate([
            np.random.exponential(params['t1'], n_direct_singlets),
            np.random.exponential(params['t3'], n_direct_triplets),
            simulate_recombination_times2(n_recombination_singlets, params['t1'], params['tr'], params['eta'],
                                          250, 1251), 
            simulate_recombination_times2(n_recombination_triplets, params['t3'], params['tr'], params['eta'],
                                          250, 1251), 
        ])    
    else:
        raise ValueError('S1 model type not understood, got this: %s' % params['s1_model'])

    # Since `times` is now sorted in (singlet, triplet), shuffle them
    np.random.shuffle(times)
    
    # Here we start taking into account detector physics: the transit time spread (simulated as normal dist.)
    times += np.random.normal(0, params['tts'], size=n_photons)
    
    # Find the bin that the photon would be in if it were sampled.
    # Now, we delete all the photons that are outside of the bin range and re-match to the bin centers
    # (Check the searchsorted documentation)
    indices = np.searchsorted(spe_t_edges_fine, times)
    indices = indices[~((indices == 0) | (indices > (len(spe_t_edges_fine) - params['max_shift'])))] - 1
    # This is the new amount of photons simulated
    if len(indices) < n_photons:
        # print('Warning: I just threw away %d photons...' % (n_photons - len(indices)))
        n_photons = len(indices)
        
    ##
    # Build instruction matrix, simulate waveforms
    ##
    # So far, we've just been simulating a bunch of photons (very many).
    # We are now going to split this into S1s: the split will be made at a random point between s1_min and s1_max.
    # `index_matrix` is a matrix split into groups forming S1s. 
    # We've got two for the two channels    
    index_matrix0, index_matrix1 = split_s1_groups(indices, len(spe_t_edges_fine) - 1, areas, channels,
                                                   **params)

    # Now, index_matrix[:, 0] contains a list of number of entries for the shift for each timestamp in bin
    n_s1 = index_matrix0.shape[1]
    
    # Remember that wv_matrix is a matrix of waveforms, each element at position i of which is shifted i samples
    s1_waveforms = np.dot(wv_matrix_list[0], index_matrix0) + np.dot(wv_matrix_list[1], index_matrix1)
    
    
    # This should work as long as the photon density is so low that the 
    # probability of many photons per bin is low (or the photons)
    # will not be added independently any more
    s1_waveforms_error = np.sqrt(np.dot(wv_err_matrix**2, index_matrix0**2 + index_matrix1**2))

    
    
    ##
    # Alignment based on 10p point
    ##
    time_matrix, t_shift = aligned_time_matrix(spe_ts, s1_waveforms)    
    return s1_waveforms, s1_waveforms_error, time_matrix, t_shift

def aligned_time_matrix(ts, wv_matrix, mode = '10p_interp'):
    """Return time matrix that would align waveforms im wv_matrix.
    As it turns out, 10p mode yields the same as 10p_interp if you use 2 ns samples in the average wf. Hmm. 
    """
    n_s1 = wv_matrix.shape[1]
    fraction_reached = np.cumsum(wv_matrix, axis=0) / np.sum(wv_matrix, axis=0)
    
    if mode == '10p':
        # Get the sample where 10% is reached by taking the sample closest to the 10% point
        distance_to_10p_point = np.abs(fraction_reached - 0.1)              
        t_shift = ts[np.argmin(distance_to_10p_point, axis=0)]
    elif mode == '10p_interp':
        # Get the sample index corresponding to the sample just before the crossing
        crossing_index = np.argmax(fraction_reached > 0.1, axis=0) - 1
        # Get fractions before and after crossing (thanks numpy indexing, I want my two hours back)
        f0 = fraction_reached[crossing_index, range(len(crossing_index))]
        f1 = fraction_reached[crossing_index + 1, range(len(crossing_index))]
        # Now add the time at that point and the interpolation
        delta_t = 2.* (0.1 - f0) / (f1 - f0)
        t_shift = ts[crossing_index] + delta_t       
    time_matrix = np.repeat(ts, n_s1).reshape(wv_matrix.shape)
    time_matrix -= t_shift[np.newaxis,:]
    return time_matrix, t_shift

def average_pulse(time_matrix, wv_matrix, **params):
    """Return average pulse, given time and waveform matrices"""
    params = get_params(params)
    h, _     = np.histogram(time_matrix, bins=spe_t_edges, weights=wv_matrix)
    # normalize within time range
    sel  = (spe_ts >= params['t_min']) & (spe_ts < params['t_max'])
    h = h / np.sum(h[sel])
    return h

def average_pulse_and_error(time_matrix, wv_matrix, wv_err_matrix, **params):
    """Return average pulse and error, given time and waveform matrices"""
    params = get_params(params)
    h, _     = np.histogram(time_matrix, bins=spe_t_edges, weights=wv_matrix)
    h_err, _ = np.histogram(time_matrix, bins=spe_t_edges, weights=wv_err_matrix)
    sel  = (spe_ts >= params['t_min']) & (spe_ts < params['t_max'])
    h_err = h_err / np.sum(h[sel])
    h = h / np.sum(h[sel])
    return h, h_err

def s1_average_pulse_model(*args, **kwargs):
    wv_matrix, wv_err_matrix, time_matrix, _ = simulate_s1_pulse(*args, **kwargs)
    model, error = average_pulse_and_error(time_matrix, wv_matrix, wv_err_matrix,**kwargs)
    return model, error

def unison_shuffled_copies(a, b):
    '''Stack overflow to the rescue'''
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

## Systematic errors

In [15]:
import itertools
import math
def s1_models_error(*args, shifts=None, mode='std', **kwargs):
    '''
    Gives the systematic error, split into two parts.
    Returns `model`, `sigma_syst`, `spe_error`
    '''
    if kwargs.get('neglect_systematic', default_params['neglect_systematic']):
        model, error = s1_average_pulse_model(*args, **kwargs)
        return model, np.zeros(len(model)), error
    
    if shifts is None:
        # Default uncertainty: in pulse model
        shifts = dict(aft=0.12)
    
    # Allow specifying a single +- amplitude of variation
    for p, shift_values in shifts.items():
        if isinstance(shift_values, (float, int)):
            shifts[p] = kwargs.get(p, default_params[p]) + np.array([-1, 0, 1]) * shift_values
    

    shift_pars = sorted(shifts.keys())
    shift_values = [shifts[k] for k in shift_pars]
    # shift_value_combs is a list of paramters that will be tried to compute the average pulse.
    # Contains all combintations: (+, 0, -) for all the parameters. ((3n)^2 for n number of parameters.)
    shift_value_combs = list(itertools.product(*shift_values))
    noshift_comb = tuple([(kwargs.get(p, default_params[p])) for p, shift_values in shifts.items()])
    noshift_index = int((len(shift_value_combs) -1) /2)
    # Check if we have the right index
    
    for i, comb in enumerate(shift_value_combs):
        if np.all([np.isclose(a,b) for a, b in zip(comb, noshift_comb)]):
            noshift_index = i
                
    
    alt_models = []
    for vs in shift_value_combs:
        kw = dict()
        kw.update(kwargs)
        for i, p in enumerate(shift_pars):
            kw[p] = vs[i]        
        alt_models.append(s1_average_pulse_model(*args, **kw))
    # The alt_models list will now contain (model, error) for each parameter combination.
    # Here, split them in twain
    alt_errs = [alt_model[1] for alt_model in alt_models]
    alt_models = [alt_model[0] for alt_model in alt_models]
    
    alt_models = np.vstack(alt_models)
    spe_err = np.vstack(alt_errs)
    model = alt_models[noshift_index]
    
    spe_err = alt_errs[noshift_index]
    sigma_sys = np.sqrt(np.std(alt_models, axis=0)**2)
    #minus = base_model - sigma_sys
    #plus = base_model + sigma_sys

    return model, sigma_sys, spe_err
    

# Real data waveforms

## Read the data

Here we read the S1 data for three (highfield) datasets: NR, ER and BG_NR. We store it in the form of a dict (keys: er, nr, bg_nr). Each dict item is an array containing the waveforms (per row).

In [16]:
print('Reading XAMS data from pickles...')

Reading XAMS data from pickles...


In [17]:
before = time.time()
xams_data = dict()
xams_data['nr'], xams_data['er'], xams_data['bg_nr'] = pickle.load(open(os.path.join(pickle_dir, 
                                                                                     'highfield_dataframes.pickle'),'rb'))
xams_data['er_0'] = pickle.load(open(os.path.join(pickle_dir, 'zerofield_dataframes_temp.pickle'), 'rb'))
xams_data['nr_l'], xams_data['er_l'] = pickle.load(open(os.path.join(pickle_dir, 'lowfield_dataframes.pickle'), 'rb'))
xams_s1s = dict()
# Get pulse waveforms to matrix rather than object column
# Also cut off a bit at the end to account for shorter waveform in high resolution pulse
max_index = len(spe_ts)
for k, d in xams_data.items():
    xams_s1s[k] = np.array([x[:max_index] for x in d['s1_pulse']])
    del d['s1_pulse']
after = time.time()
print('Read data in %.2f seconds' % (after - before))

Read data in 23.14 seconds


For now use 0.28 +- 0.12 as uncertainty of the fit

## Alignment, averaging, bootstrapping

In [18]:
def real_s1_wv(**params):
    """Return average S1 waveform, number of S1s it was constructed from"""
    params = get_params(params)
    
    areas    = xams_data[params['dset']]['s1'].values
    energies = xams_data[params['dset']]['e_ces'].values
    mask = ((params['s1_min'] < areas) & (areas < params['s1_max']) &
            (energies >= params['e_min']) & (energies < params['e_max']))

    n_data_s1s = mask.sum()
    wvs = xams_s1s[params['dset']][mask].T
    tmat, _ = aligned_time_matrix(spe_ts, wvs)
    real_s1_avg =  average_pulse(tmat, wvs, **params)
    
    return real_s1_avg, n_data_s1s

In [19]:
def real_s1_wv_sigma(**params):
    """Take data S1s, bootstrap sample, then check what the variance is"""
    params = get_params(params)
    if params['stored_stat']:
        if params['stored_stat_value'] is not None:
            return params['stored_stat_value']
        else:
            print('Warning: stored_stat not found, will revert to computing again...')
    bootstrap_trials = params['bootstrap_trials']
    time_matrix, wv_matrix = real_s1_wv_matrix(**params)
    n_s1s = wv_matrix.shape[1]
    waveform_templates = np.zeros((len(spe_ts), bootstrap_trials))

    for i in range(bootstrap_trials):
        new_indices = np.random.randint(n_s1s, size=n_s1s)

        waveform_templates[:, i] = average_pulse(time_matrix[:, new_indices], 
                                                 wv_matrix[:, new_indices], **params)
        
    
    return np.std(waveform_templates, axis=1)

In [20]:
def real_s1_wv_matrix(**params):
    """Return the aligned time matrix and waveform matrix."""
    params = get_params(params)
    
    areas = xams_data[params['dset']]['s1'].values
    mask = (params['s1_min'] < areas) & (areas < params['s1_max'])

    n_data_s1s = mask.sum()
    wvs = xams_s1s[params['dset']][mask].T
    tmat, _ = aligned_time_matrix(spe_ts, wvs)
        
    return tmat, wvs

# Model-data comparison

## Sigma and residuals

In [1]:
def residuals(ydata, model, syst_err, spe_err, **params):
    params = get_params(params)
    sigma = get_sigma(model, syst_err, spe_err, **params)
    if 0. in sigma:
        zero_positions = np.where(sigma == 0)
        print('Warning: found zero in error array at positions: ', zero_positions)
        print('Replacing with infinite error instead...')
        for pos in zero_positions:
            sigma[pos] = np.inf
    return (ydata - model) / sigma

def get_sigma(model, syst_err, spe_err, **params):
    '''
    Combine the syst. errors with stat error.
    '''
    params = get_params(params)
    if params['neglect_statistical']:
        print('Neglecting statistical error...')
        return np.sqrt(syst_err**2 + spe_err**2)
    sigma_stat = real_s1_wv_sigma(**params)
    sigma = np.sqrt(syst_err**2 + spe_err**2 + sigma_stat**2 + 
                    params['error_offset']**2) #  + (model * params['error_pct'])**2
    return sigma

## Plotting

In [22]:
def comparison_plot(ydata, model, syst_err, spe_err, log=False, **params):
    params = get_params(params)
    sigmas = get_sigma(model, syst_err, spe_err, **params)

    # large subplot
    ax2 = plt.subplot2grid((3,1), (2,0))
    ax1 = plt.subplot2grid((3,1), (0,0), rowspan=2, sharex=ax2)

    #f, (ax1, ax2) = plt.subplots(2, sharex=True)
    plt.sca(ax1)
    # plt.fill_between(spe_ts, minus, plus, alpha=0.5, linewidth=0, step='mid')
    plt.fill_between(spe_ts, model - sigmas, model + sigmas,
                     alpha=0.5, linewidth=0, step='mid')
    plt.plot(spe_ts, model, linestyle='steps-mid', label='Model')
    plt.plot(spe_ts, ydata, marker='.', linestyle='', markersize=3, c='k', label='Observed')

    plt.grid(alpha=0.1, linestyle='-', which='both')
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.ylabel("Fraction of amplitude")
    plt.axhline(0, c='k', alpha=0.5)
    leg = plt.legend(loc='upper right', numpoints=1)
    leg.get_frame().set_linewidth(0.0)
    leg.get_frame().set_alpha(0.5)
    if log:
        plt.yscale('log')
        plt.ylim(1e-5, 1e-1)
    else:
        plt.ylim(0, None)

    # Add residuals
    plt.sca(ax2)
    plt.subplot2grid((3,1), (2,0), sharex=ax1)
    plt.xlim(params['t_min'], params['t_max'])

    res = (ydata - model) / sigmas # residuals(ydata, minus, base, plus, **params)
    
    plt.plot(spe_ts, res,
             linestyle='--', marker='x', c='k', markersize=3)
    # plt.ylim(-3, 3)
    plt.grid(which='both', linestyle='-', alpha=0.1)
    plt.axhline(0, c='k', alpha=0.5)

    plt.ylabel("Residual")
    plt.xlabel("Time since alignment point")

    plt.tight_layout()
    plt.gcf().subplots_adjust(0,0,1,1,0,0)

def comparison_plot_2(ydata, model, syst_err, spe_err, plot_residual = True, **params):
    params = get_params(params)
    sigmas = get_sigma(model, syst_err, spe_err, **params)
    res = (ydata - model) / sigmas 

    plt.fill_between(spe_ts, model - sigmas, model + sigmas,
                     alpha=0.5, linewidth=0, step='mid')
    plt.plot(spe_ts, model, linestyle='steps-mid', label='Model')
    plt.plot(spe_ts, ydata, marker='.', linestyle='', markersize=3, c='k', label='Observed')
    plt.yscale('log')
    plt.ylim(2e-5, 1e-1)
    plt.ylabel("Fraction of amplitude")
    plt.xlabel('Time (ns)')
    for _l in (params['t_min'], params['t_max']):
        plt.axvline(_l, ls='dotted', color='black')
    if plot_residual:
        plt.twinx()
        plt.plot(spe_ts, np.abs(res), color='red')
        plt.ylabel('Residual / error')
        plt.ylim(0)
    plt.xlim(params['t_min'] - 20, params['t_max'] + 50)
    
    res = res[(spe_ts >= params['t_min']) & (spe_ts < params['t_max'])]
    chi2 = sum(res**2) / len(spe_ts[(spe_ts >= params['t_min']) & (spe_ts < params['t_max'])])
    print('chi2 = %f' % chi2)
    
def plot_model(plot_type = 0, figname = None, plot_residual = True, verbose=True, **params):
    params = get_params(params)
    if verbose:
        print(params)
    ydata, _ = real_s1_wv(**params)
    model, syst_err, spe_err = s1_models_error(**params)
    if plot_type == 0 or plot_type == 1:
        comparison_plot(ydata, model, syst_err, spe_err, **params)
        if figname is not None:
            plt.savefig('figs/' + figname + '_1.png', bbox_inches = 'tight', dpi = 400)
        plt.show()
    if plot_type == 0 or plot_type == 2:
        comparison_plot_2(ydata, model, syst_err, spe_err, plot_residual = plot_residual, **params)
        if figname is not None:
            plt.savefig('figs/' + figname + '_2.png', bbox_inches = 'tight', dpi = 400)

        plt.show()
    return



## Residuals function

In [23]:
def gof(verbose=True, **params):
    '''
    Get the value to minimize given the parameters
    '''
    params = get_params(params)
    # Do not allow unphysical values
    if params['t1'] < 0 or params['t3'] < 0 or not (0 <= params['fs'] <= 1):
        result = float('inf')
    else:
        ydata, _ = real_s1_wv(**params)
        model, syst_err, spe_err = s1_models_error(**params)
        res = residuals(ydata, model, syst_err, spe_err, **params)
        assert len(res) == len(spe_ts)
        res = res[(spe_ts >= params['t_min']) & (spe_ts < params['t_max'])]
        # Computed chi2 over NDF
        result = 1/len(res) * np.sum(res**2)
    if verbose:
        print('gof={gof}, fs={fs}, t1={t1}, t3={t3}, tts={tts}'.format(gof=result, **params))
    return result

In [25]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def minimize_it(param_names, starting_values, direc, **params):
    optresult = optimize.minimize(
        lambda x: gof(**merge_two_dicts(params, {par : x[i] for i, par in enumerate(param_names)})),
        starting_values,
        options=dict(maxfev=1000, direc=direc),
        method='Powell',
    )
    return optresult

# Automatic functions

In [29]:
def produce_settings_dicts(scan_params, lower_bounds, upper_bounds, step_sizes, block_size=int(1e6), verbose=True, **p):
    '''
    Build a number of settings dicts in an array. These settings scan over the bounds values.
    '''
    x = []
    nsteps = [int(np.round((u - l)/s)) + 1 for u, l, s in zip(upper_bounds, lower_bounds, step_sizes)]
    for param, l, u, ns in zip(scan_params, lower_bounds, upper_bounds, nsteps):
        x.append(np.linspace(l, u, ns))
    n_sim  = int(np.product(nsteps))
    x = np.meshgrid(*x)
    par_vals = [_x.flatten() for _x in x]
    dicts = []
    for i in range(n_sim):
        for j, par in enumerate(scan_params):
            p[par] = par_vals[j][i]
        dicts.append(deepcopy(p))
    dicts = np.array(dicts)
    if verbose: print('Loaded %d settings.' % len(dicts))
    if len(dicts) <= block_size:
        return dicts
    else:
        n_blocks = int(len(dicts) / block_size)
        return np.array_split(dicts, n_blocks)

In [30]:
def process_settings(dicts):
    for p in dicts:
        p['chi2'] = gof(**p)
    return dicts

In [31]:
def dump_settings_pickles(x, dirname, base_name):
    fns = []
    for i, _x in enumerate(x):
        if not os.path.exists(dirname):
            os.makedirs(dirname)
        fn = os.path.join(dirname, base_name + '_%03d.pickle' % i)
        fns.append(fn)
        with open(fn, 'wb') as f:
            pickle.dump(_x, f)
    print('Dumping done.')
    return fns

In [32]:
def single_pickle_read(fn):
    with open(fn, 'rb') as f:
        dicts = pickle.load(f)
    return dicts

def single_pickle_dump(fn, dicts):
    with open(fn, 'wb') as f:
        pickle.dump(dicts, f)
    return

In [ ]:
def split_settings(dict_list, nsamples):
    n_lists = int(np.floor(len(dict_list) / nsamples))
    return np.array_split(dict_list, n_lists)

# Result analysis

These functions are meant to analyze the data post-processing, assuming that all the relevant information is in the procesed data files (i.e. dicts). This means nothing needs to be recomputed, yay!

In [ ]:
def plot_model_manual(plot_type = 0, figname = None, **params):
    '''
    Plot the model assuming that all needed parameters are in `params` (such as syst error etc.)
    '''
    params = get_params(params)
    ydata, model, syst_err, spe_err = (params['ydata'], params['model'], params['syst_err'], params['spe_err'])
    if plot_type == 0 or plot_type == 1:
        comparison_plot(**params)
        if figname is not None:
            plt.savefig('figs/' + figname + '_1.png', bbox_inches = 'tight', dpi = 400)
        plt.show()
    if plot_type == 0 or plot_type == 2:
        comparison_plot_2(**params)
        if figname is not None:
            plt.savefig('figs/' + figname + '_2.png', bbox_inches = 'tight', dpi = 400)

        plt.show()
    return

In [1]:
def check_errors(**p):
    """
    Plot the errors by type as function of time and check where the sigma contribution lies.
    Nice for nailing down the error definitions.
    """
    p = get_params(p)
    for key in ['spe_err', 'stored_stat_value', 'syst_err']:
        plt.plot(spe_ts, p[key], label= key)
    plt.plot(spe_ts, np.ones(len(spe_ts)) * p['error_offset'], label='error_offset')
    sigmas = get_sigma(**p)
    plt.plot(spe_ts, sigmas, label='Total error')
    plt.legend(loc=(1.05,0))

    for t in [p['t_min'], p['t_max']]:
        plt.axvline(t, ls='--', lw=1, color='black')
    plt.xlim(p['t_min'] -10, p['t_max'] + 25)
    plt.show()
    
    
    deltas = np.abs(p['model'] - p['ydata'])
    plt.plot(spe_ts, deltas)
    plt.plot(spe_ts, sigmas)
    peak = (spe_ts >= -10) & (spe_ts < 30)
    tail = (spe_ts >= 30) & (spe_ts < 125)
    inrange = (spe_ts >= -10) & (spe_ts < 125)
    print('Chi2 in peak: (-10 to 30 ns): %.2f' % (np.average(deltas[peak]**2 / sigmas[peak]**2)))
    print('Chi2 in tail: (30 to 125 ns): %.2f' % (np.average(deltas[tail]**2 / sigmas[tail]**2)))
    print('Chi2 total: %.2f' % (np.average(deltas[inrange]**2 / sigmas[inrange]**2)))
    plt.show()

In [ ]:
def gof_manual(ydata, model, syst_err, spe_err, verbose=True, **params):
    params = get_params(params)
    res = residuals(ydata, model, syst_err, spe_err, **params)
    assert len(res) == len(spe_ts)
    res = res[(spe_ts >= params['t_min']) & (spe_ts < params['t_max'])]
    # Computed chi2 over NDF
    result = 1/len(res) * np.sum(res**2)
    if verbose: print('gof={gof}, fs={fs}, t1={t1}, t3={t3}, tts={tts}'.format(gof=result, **params))
    return result

# Mock waveforms production

New jargon warning: 'mock data' is simulated data that is meant to simulate the actual amount of waveforms.

In [ ]:
def mock_s1_wv_sigma(time_matrix, wv_matrix, **params):
    """Take data S1s, bootstrap sample, then check what the variance is"""
    params = get_params(params)
    bootstrap_trials = params['bootstrap_trials']
    n_s1s = wv_matrix.shape[1]
    waveform_templates = np.zeros((len(spe_ts), bootstrap_trials))

    for i in range(bootstrap_trials):
        new_indices = np.random.randint(n_s1s, size=n_s1s)

        waveform_templates[:, i] = average_pulse(time_matrix[:, new_indices], 
                                                 wv_matrix[:, new_indices], **params)
        
    return np.std(waveform_templates, axis=1)

In [ ]:
def build_mock_data(verbose=True, return_stored = True, **params):
    '''
    Mock data is simulated data with just the right statistics given the data.
    Arguments: parameters specifying the dataset (dset, e_min, e_max), plus the true values (i.e. the fitted values).
    '''
#     params = get_params(params)
    d = xams_data[params['dset']]
    d = d[(d['e_ces'] >= params['e_min']) & (d['e_ces'] < params['e_max']) ]
    n_s1s = len(d)
    if verbose:
        print("Selected data from %s, energy from %d to %d" % (params['dset'], params['e_min'], params['e_max']))
        print("Selected %d waveforms" % (n_s1s))
    
    # Take a generous sample of photons to eneter into the simulation
    n_photons_guess = int(1.5 * sum(d['cs1']))
    s1_waveforms, s1_waveforms_error, time_matrix, t_shift  = simulate_s1_pulse(n_photons = n_photons_guess, **params)
    # Cut off the excess waveforms to exactly match stats
    if len(t_shift) < n_s1s:
        raise ValueError("Number of S1s incorrect, how is possible?")
    s1_waveforms = s1_waveforms[:, :n_s1s]
    s1_waveforms_error = s1_waveforms_error[:, :n_s1s]
    t_shift = t_shift[:n_s1s]
    time_matrix = time_matrix[:, :n_s1s]
    avg = average_pulse(time_matrix, s1_waveforms)
    if return_stored:
        stored_stat = mock_s1_wv_sigma(time_matrix, s1_waveforms)
        return avg, stored_stat
    else:
        return avg